In [16]:
import requests
import re

def main():
    # 1) 세션(Session) 객체 생성
    # 세션을 사용해야 쿠키가 자동으로 저장되어 다음 요청에 함께 전송됩니다
    s = requests.Session()

    # 2) 브라우저 유사한 User-Agent 설정 (User-Agent 검사를 우회하기 위함)
    # Chrome, Firefox 등 다양한 User-Agent 중 하나로 지정합니다
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    })

    # 3) /level3 페이지를 먼저 요청하여 세션 쿠키와 HTML 내 js_token 획득
    url_level3 = "http://127.0.0.1:5000/level3"
    r = s.get(url_level3)
    if r.status_code != 200:
        print("level3 페이지 요청 실패: ", r.status_code, r.text)
        return

    # HTML에서 토큰 추출 (템플릿 예시에 따라 정규식 등을 활용)
    # level3.html 내부에서:  const token = "{{ session['js_token'] }}";
    # 이런 식으로 들어있을 가능성이 높습니다
    match = re.search(r'const\s+token\s*=\s*["\']([^"\']+)["\']', r.text)
    if not match:
        print("토큰을 찾을 수 없습니다. HTML 구조를 확인하세요.")
        return

    js_token = match.group(1)
    print("획득한 js_token:", js_token)

    # 4) /protected_api/articles 요청
    # Referer 헤더 추가 + GET 파라미터에 token 추가
    url_articles = "http://127.0.0.1:5000/protected_api/articles"
    params = {
        "page": 1,
        "pageSize": 20,
        "token": js_token  # 자바스크립트 검증(세션 토큰)
    }
    # Referer에 level3 문자열이 반드시 들어가도록 설정합니다
    headers = {
        "Referer": "http://127.0.0.1:5000/level3"
    }
    
    r2 = s.get(url_articles, params=params, headers=headers)

    if r2.status_code == 200:
        print("정상 응답:", r2.json())  # JSON 데이터 출력
    else:
        print("차단된 응답:", r2.status_code, r2.text)

if __name__ == "__main__":
    main()


획득한 js_token: c79b2bfe2e8b46c43d2fde40823abcea
정상 응답: [{'article_area_name': '26B', 'article_area_size': 16, 'article_deal_or_warrant_price': '2억 2,000', 'article_deal_price': 22000, 'article_desc': '역세권 즉시입주가능 물건', 'article_detail_desc': '- 선릉역 도보5분이내\n\n- 위치가 좋아 공실걱정없는 임차수요\n\n- 24시간 경비순찰/ 안정적인 치안\n\n- 현 임차인 거주 / 매매시 바로 입주가능\n\n---------------------------------------------------------------------\n매물담당 : ☎ 02-501-4568\n---------------------------------------------------------------------\n\n▶ 담당자에게 전화주시면 사전 예약 후 직접 안내해 드리겠습니다◀\n\u200b\u200b\nㆍㆍ 더 퍼플 부동산 ㆍㆍ \n\u200b\n삶의 여정과 아름다운 순간들이\n고스란히 담긴 공간은  당신의 미래를 만듭니다. \n새로운 시작을 꿈꾸시는 분들께 \n더 퍼플이 함께하겠습니다. \n\u200b\u200b\n고급주거, 하이엔드주거 전문 부동산입니다.\n고객님의 프라이버시와 만족을 최우선으로 생각합니다.\n\n성북동 매물 다수보유 및 2024년 다수계약체결\n성심성의껏 상담해 드리겠습니다.\n\u200b\n찾으시는 매물의 조건을 말씀주시면\n최고의 매물을 찾아 소개드리겠습니다.\n---------------------------------------------------------------------\n블로그: https://m.blog.naver.com/bora_house\n유튜브: 더 퍼플 부동산 검색🔎 \n인스타그램: @thepurple_realty', 'article_direc

In [ ]:
import requests
import time

cookies = {
    'session': '.eJwtjs0KgkAURl_lclcKok7-ga6iRYHZJsGljHnNyUllRimK3r0h4tsdzoHvjXWnSPeYdlxqclDMNW9bgzSmyDaJ65sxdPCmaxp5I6nFdFGrUVdNquZXGhejFtNLSMm9yPXBqsTYTg8NpxKY6TMwIA4zeMahDdt5llRRk4vFi4LEDWKw8kNZHB2QYiDY02WYbNj1arqTxwL2u-DDmXdciX-Cny9A-jfs.Z25o2A.7vdzkqD6fnytvBviNIqnbaHE5SA',
}
 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}

params = {
    'page': '2',
    'pageSize': '20',
}

session = requests.Session()
session.headers.update(headers)
session.cookies.update(cookies)

for i in range(100):
    response = session.get('http://127.0.0.1:5000/protected_api/articles', params=params, cookies=cookies, headers=headers)
    print(response)
    time.sleep(1)

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('http://127.0.0.1:5000/level3')
time.sleep(1)

for i in range(1, 100):
    driver.get(f'http://127.0.0.1:5000/protected_api/articles?page={i}&pageSize=20')
    driver.
    time.sleep(5)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF7CA3AFB05+28789]
	(No symbol) [0x00007FF7CA3186E0]
	(No symbol) [0x00007FF7CA1B592A]
	(No symbol) [0x00007FF7CA18F505]
	(No symbol) [0x00007FF7CA236477]
	(No symbol) [0x00007FF7CA24EF42]
	(No symbol) [0x00007FF7CA22F1E3]
	(No symbol) [0x00007FF7CA1FA938]
	(No symbol) [0x00007FF7CA1FBAA1]
	GetHandleVerifier [0x00007FF7CA6E933D+3410093]
	GetHandleVerifier [0x00007FF7CA6FE7DD+3497293]
	GetHandleVerifier [0x00007FF7CA6F2A73+3448803]
	GetHandleVerifier [0x00007FF7CA477BBB+848171]
	(No symbol) [0x00007FF7CA323C3F]
	(No symbol) [0x00007FF7CA31F6E4]
	(No symbol) [0x00007FF7CA31F87D]
	(No symbol) [0x00007FF7CA30ED49]
	BaseThreadInitThunk [0x00007FFF69E07374+20]
	RtlUserThreadStart [0x00007FFF6AFBCC91+33]
